<a href="https://colab.research.google.com/github/BellaBeautiful4092023/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [808 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,082 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ub

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price),2)
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY date_built
    """).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           307908.86|
|      2013|           299999.39|
|      2014|           299073.89|
|      2012|           298233.42|
|      2016|           296050.24|
|      2010|           296800.75|
|      2011|            302141.9|
|      2017|           296576.69|
+----------+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price),2)
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    """).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|            288770.3|
|      2013|           295962.27|
|      2014|           290852.27|
|      2012|           293683.19|
|      2016|           290555.07|
|      2010|           292859.62|
|      2011|           291117.47|
|      2017|           292676.79|
+----------+--------------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
    SELECT date_built, ROUND(AVG(price),2)
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living > 2000
    GROUP BY date_built
    """).show()



+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           297609.97|
|      2013|           303676.79|
|      2014|           297619.46|
|      2012|           307539.97|
|      2016|            293965.1|
|      2010|           285010.22|
|      2011|           276553.81|
|      2017|           280527.62|
+----------+--------------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT format_number(AVG(price), 2) AS average_price, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
| 1,026,669.50| 100|
| 1,061,201.42|  99|
| 1,053,739.33|  98|
| 1,129,040.15|  97|
| 1,017,815.92|  96|
| 1,054,325.60|  95|
| 1,033,536.20|  94|
| 1,026,006.06|  93|
|   970,402.55|  92|
| 1,137,372.73|  91|
| 1,062,654.16|  90|
| 1,107,839.15|  89|
| 1,031,719.35|  88|
| 1,072,285.20|  87|
| 1,070,444.25|  86|
| 1,056,336.74|  85|
| 1,117,233.13|  84|
| 1,033,965.93|  83|
| 1,063,498.00|  82|
| 1,053,472.79|  81|
+-------------+----+
only showing top 20 rows

--- 2.1804444789886475 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT format_number(AVG(price), 2) AS average_price, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|average_price|view|
+-------------+----+
| 1,026,669.50| 100|
| 1,061,201.42|  99|
| 1,053,739.33|  98|
| 1,129,040.15|  97|
| 1,017,815.92|  96|
| 1,054,325.60|  95|
| 1,033,536.20|  94|
| 1,026,006.06|  93|
|   970,402.55|  92|
| 1,137,372.73|  91|
| 1,062,654.16|  90|
| 1,107,839.15|  89|
| 1,031,719.35|  88|
| 1,072,285.20|  87|
| 1,070,444.25|  86|
| 1,056,336.74|  85|
| 1,117,233.13|  84|
| 1,033,965.93|  83|
| 1,063,498.00|  82|
| 1,053,472.79|  81|
+-------------+----+
only showing top 20 rows

--- 1.2170732021331787 seconds ---


In [12]:
home_sales_df.write.partitionBy('date_built').jdbc

<bound method DataFrameWriter.jdbc of <pyspark.sql.readwriter.DataFrameWriter object at 0x7ddf8013ff40>>

In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [14]:
# 11. Read the formatted parquet data.
p_df_p =spark.read.parquet('home_sales_partitioned')

In [15]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales')

In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT format_number(AVG(price), 2) AS average_price, view
FROM home_sales
WHERE price >= 350000
GROUP BY view
ORDER BY view DESC""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
| 1,026,669.50| 100|
| 1,061,201.42|  99|
| 1,053,739.33|  98|
| 1,129,040.15|  97|
| 1,017,815.92|  96|
| 1,054,325.60|  95|
| 1,033,536.20|  94|
| 1,026,006.06|  93|
|   970,402.55|  92|
| 1,137,372.73|  91|
| 1,062,654.16|  90|
| 1,107,839.15|  89|
| 1,031,719.35|  88|
| 1,072,285.20|  87|
| 1,070,444.25|  86|
| 1,056,336.74|  85|
| 1,117,233.13|  84|
| 1,033,965.93|  83|
| 1,063,498.00|  82|
| 1,053,472.79|  81|
+-------------+----+
only showing top 20 rows

--- 0.5332098007202148 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("Yikes!, it looks like home_sales is still cached.")
else:
  print("Yay!, home_sales is no longer cached!")


Yay!, home_sales is no longer cached!
